# Notebook 4 : Silver 3 - ML Prédiction (PySpark MLlib)

**Durée** : 20 minutes  
**Lakehouse** : Lakehouse_silver  
**Objectif** : Entraîner un modèle de prédiction de consommation J+1

## Cellule 1 : Import MLlib

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F

print("✅ MLlib importé")

## Cellule 2 : Chargement données Silver enriched

In [ ]:
df = spark.table("Lakehouse_silver.silver.consumption_enriched")
print(f"📊 Données chargées : {df.count()} lignes")
df.printSchema()

## Cellule 3 : Préparation features pour ML

In [ ]:
# Features pertinentes
features_cols = [
    "hour_of_day",
    "day_of_week",
    "temperature_c",
    "wind_speed_ms",
    "consumption_lag_1d",
    "consumption_lag_7d",
    "baseline_7d_mw",
    "price_eur_mwh"
]

target_col = "avg_consumption_mw"

# Filtrer les NULL (après lags)
df_ml = df.select(features_cols + [target_col, "hour", "site_id"]).na.drop()

print(f"📊 Dataset ML : {df_ml.count()} lignes (après suppression NULL)")
df_ml.show(5)

## Cellule 4 : VectorAssembler + StandardScaler

In [ ]:
assembler = VectorAssembler(
    inputCols=features_cols,
    outputCol="features_raw"
)

scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withStd=True,
    withMean=True
)

print("✅ VectorAssembler + Scaler configurés")

## Cellule 5 : Split Train/Test (80/20 temporel)

In [ ]:
# Split temporel : 80% premiers jours = train, 20% derniers jours = test
dates_sorted = df_ml.select("hour").distinct().orderBy("hour").collect()
split_idx = int(len(dates_sorted) * 0.8)
split_date = dates_sorted[split_idx]["hour"]

train_df = df_ml.filter(F.col("hour") < split_date)
test_df = df_ml.filter(F.col("hour") >= split_date)

print(f"📊 Train : {train_df.count()} lignes (80%)")
print(f"📊 Test  : {test_df.count()} lignes (20%)")
print(f"📅 Date de split : {split_date}")

## Cellule 6 : Entraînement LinearRegression

In [ ]:
lr = LinearRegression(
    featuresCol="features",
    labelCol=target_col,
    predictionCol="prediction",
    maxIter=100,
    regParam=0.01
)

pipeline = Pipeline(stages=[assembler, scaler, lr])

print("🚀 Entraînement du modèle...")
model = pipeline.fit(train_df)
print("✅ Modèle entraîné")

# Coefficients
lr_model = model.stages[-1]
print(f"\n📊 Intercept : {lr_model.intercept:.4f}")
print(f"📊 Nombre de coefficients : {len(lr_model.coefficients)}")

## Cellule 7 : Prédictions sur test

In [ ]:
predictions = model.transform(test_df)

predictions.select(
    "hour", 
    "site_id", 
    target_col, 
    "prediction"
).orderBy("hour").show(20)

## Cellule 8 : Évaluation (RMSE, MAE, R²)

In [ ]:
evaluator_rmse = RegressionEvaluator(
    labelCol=target_col,
    predictionCol="prediction",
    metricName="rmse"
)

evaluator_mae = RegressionEvaluator(
    labelCol=target_col,
    predictionCol="prediction",
    metricName="mae"
)

evaluator_r2 = RegressionEvaluator(
    labelCol=target_col,
    predictionCol="prediction",
    metricName="r2"
)

rmse = evaluator_rmse.evaluate(predictions)
mae = evaluator_mae.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)

print("\n" + "="*50)
print("📊 MÉTRIQUES DU MODÈLE")
print("="*50)
print(f"RMSE (Root Mean Square Error) : {rmse:.4f} MW")
print(f"MAE  (Mean Absolute Error)    : {mae:.4f} MW")
print(f"R²   (Coefficient determination): {r2:.4f}")
print("="*50)

if r2 > 0.7:
    print("✅ Modèle performant (R² > 0.7)")
elif r2 > 0.5:
    print("⚠️ Modèle acceptable (R² entre 0.5 et 0.7)")
else:
    print("❌ Modèle faible (R² < 0.5)")

## Cellule 9 : Analyse des erreurs

In [ ]:
# Calculer erreur absolue
predictions = predictions.withColumn(
    "error_abs",
    F.abs(F.col(target_col) - F.col("prediction"))
)

error_stats = predictions.select(
    F.mean("error_abs").alias("mean_error"),
    F.stddev("error_abs").alias("std_error"),
    F.max("error_abs").alias("max_error"),
    F.min("error_abs").alias("min_error")
).collect()[0]

print("\n📊 STATISTIQUES DES ERREURS")
print("="*50)
print(f"Erreur moyenne    : {error_stats['mean_error']:.4f} MW")
print(f"Écart-type        : {error_stats['std_error']:.4f} MW")
print(f"Erreur maximale   : {error_stats['max_error']:.4f} MW")
print(f"Erreur minimale   : {error_stats['min_error']:.4f} MW")
print("="*50)

# Top 10 pires prédictions
print("\n⚠️ Top 10 pires prédictions :")
predictions.select(
    "hour", 
    "site_id", 
    target_col, 
    "prediction", 
    "error_abs"
).orderBy(F.desc("error_abs")).show(10)

## Cellule 10 : Comparaison avec baseline naïve

In [ ]:
# Baseline naïve : prédiction = consommation J-1
predictions_baseline = predictions.withColumn(
    "prediction_baseline",
    F.col("consumption_lag_1d")
)

rmse_baseline = RegressionEvaluator(
    labelCol=target_col,
    predictionCol="prediction_baseline",
    metricName="rmse"
).evaluate(predictions_baseline)

print("\n📊 COMPARAISON MODÈLE vs BASELINE NAÏVE")
print("="*50)
print(f"RMSE Modèle ML      : {rmse:.4f} MW")
print(f"RMSE Baseline (J-1) : {rmse_baseline:.4f} MW")
improvement = ((rmse_baseline - rmse) / rmse_baseline * 100)
print(f"Amélioration        : {improvement:.1f}%")
print("="*50)

if rmse < rmse_baseline:
    print("✅ Le modèle ML est meilleur que la baseline naïve")
else:
    print("❌ Le modèle ML n'améliore pas la baseline")

## Cellule 11 : Sauvegarde prédictions

In [ ]:
# Sauvegarder toutes les prédictions (train + test)
all_predictions = model.transform(df_ml)

all_predictions.write.mode("overwrite").format("delta").saveAsTable("silver.consumption_predictions")

print(f"✅ Prédictions sauvegardées : {all_predictions.count()} lignes")
print("📍 Table : silver.consumption_predictions")

## Cellule 12 : Résumé

### ✅ ML Prédiction terminé

**Modèle entraîné** :
- Algorithme : Linear Regression
- Features : 8 (heure, jour, météo, lags, baseline, prix)
- Train/Test : 80/20 (split temporel)

**Performance** :
- RMSE, MAE, R² affichés ci-dessus
- Amélioration vs baseline naïve calculée

**Prédictions sauvegardées** :
- 📍 Table : silver.consumption_predictions

**💡 Pourquoi PySpark était obligatoire ?**
- ❌ Spark SQL ne peut pas faire de ML (pas de MLlib en SQL)
- ✅ PySpark seul permet VectorAssembler, LinearRegression, Pipeline
- ✅ PySpark permet évaluation avec métriques (RMSE, MAE, R²)

➡️ **Prochaine étape** : Agrégations business dans Gold (Notebook 5)